In [1]:
import os
# 获取当前工作目录路径
current_dir = os.getcwd()
import pandas as pd

In [3]:
#方法1.1：用TF-IDF直接对中国职业名的英文翻译和美国O*NET职业名一一匹配
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity

# Load the result.csv file
result_file = os.path.join(current_dir, 'result中英版 - 工作表1.csv')
result_df = pd.read_csv(result_file,encoding='gbk')

# Load the O*NET job data (from CSV file)
onet_file = os.path.join(current_dir, 'filtered_occupation_data.csv')
onet_jobs_df = pd.read_csv(onet_file)

# Preview the result file and O*NET job data to check column names
print(result_df.head())
print(onet_jobs_df.head())

# Assuming the 'Title' column in the result file contains the job titles (adjust if necessary)
result_jobs = result_df['Title'].dropna()  # Remove any NaN values from the D column

# Assuming O*NET job titles are in 'Job_Title' and O*NET job codes are in 'SOC_code'
onet_jobs = onet_jobs_df['Title'].dropna()  # Remove any NaN values from O*NET job titles
onet_codes = onet_jobs_df['Code'].dropna()  # The O*NET job codes

# Step 1: Create a TF-IDF vectorizer to transform the job titles into vector form
vectorizer = TfidfVectorizer(stop_words='english')

# Step 2: Combine the result job titles and O*NET job titles into one list
all_jobs = list(result_jobs) + list(onet_jobs)

# Step 3: Fit and transform the combined job titles
tfidf_matrix = vectorizer.fit_transform(all_jobs)

# Step 4: Split the TF-IDF matrix into two parts: one for result jobs and one for O*NET jobs
result_tfidf = tfidf_matrix[:len(result_jobs)]
onet_tfidf = tfidf_matrix[len(result_jobs):]

# Step 5: Compute cosine similarity between result jobs and O*NET jobs
cosine_similarities = cosine_similarity(result_tfidf, onet_tfidf)

# Step 6: Find the most similar O*NET job for each result job
best_matches = []
for idx, row in enumerate(cosine_similarities):
    best_match_idx = row.argmax()  # Index of the highest cosine similarity
    best_match_score = row[best_match_idx]  # The similarity score
    best_match_onet_job = onet_jobs.iloc[best_match_idx]  # The most similar O*NET job name
    best_match_onet_code = onet_codes.iloc[best_match_idx]  # The O*NET job code
    best_matches.append((result_df.iloc[idx, 0], result_jobs.iloc[idx], best_match_onet_job, best_match_onet_code, best_match_score))

# Step 7: Create a DataFrame for the results
matches_df = pd.DataFrame(best_matches, columns=['Result_Job_Code', 'Result_Job', 'ONet_Job', 'ONet_Code', 'Similarity_Score'])

# 保存结果到CSV文件
matches_df.to_csv("matched_jobs_name_TFIDF_filtered.csv", index=False)

# 打印前几行结果
print(matches_df.head())


         Code ChineseTitle                                     Unnamed: 2  \
0  1-01-00-00   中国共产党机关负责人              在中国共产党中央和地方各级机关及其工作机构中，担任领导职务的人员。   
1  1-02-01-00    国家权力机关负责人      在各级人民代表大会常务委员会及其工作机构中，担任领导职务并具有决策、管理权的人员。   
2  1-02-02-00    国家行政机关负责人           在各级国家行政机关及其工作机构中，担任领导职务并具有决策、管理权的人员。   
3  1-02-03-00    人民政协机关负责人             在各级人民政协及其工作机构中，担任领导职务并具有决策、管理权的人员。   
4  1-02-04-01      人民法院负责人  在最高人民法院、地方各级人民法院以及专门人民法院中，担任领导职务并具有决策、管理权的人员。   

                                               Title  \
0               Head of the Communist Party of China   
1                         Head of state power agency   
2                Head of state administrative agency   
3  Head of the People's Political Consultative Co...   
4                 person in charge of people's court   

                                         Description  
0  Personnel holding leadership positions in the ...  
1  Personnel who hold leadership positions and ha...  
2  Personnel who hold leadership po

In [4]:
#方法1.2：用TF-IDF根据中美职业定义对中国职业名和美国O*NET职业名一一匹配
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity
import nltk
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
import string


# 设置 NLTK 数据路径，确保正确加载已下载的资源
nltk.data.path.append(r"C:\Users\for30\AppData\Roaming\nltk_data")  


# 停用词
stop_words = set(stopwords.words('english'))

# 文本预处理函数
def preprocess_text(text):
    if isinstance(text, str):  # 确保是字符串
        # 转小写，去除标点，分词
        text = text.lower()
        tokens = word_tokenize(text)
        # 去除停用词和标点符号
        filtered_tokens = [word for word in tokens if word not in stop_words and word not in string.punctuation]
        return ' '.join(filtered_tokens)
    return ""

# 预处理result.csv和O*NET的Description
result_df['Processed_Description'] = result_df['Description'].apply(preprocess_text)
onet_jobs_df['Processed_Description'] = onet_jobs_df['Description'].apply(preprocess_text)

# 使用TfidfVectorizer转换文本
vectorizer = TfidfVectorizer()
# 合并所有的Description进行向量化
all_descriptions = result_df['Processed_Description'].tolist() + onet_jobs_df['Processed_Description'].tolist()

# 计算TF-IDF矩阵
tfidf_matrix = vectorizer.fit_transform(all_descriptions)

# 分割result和O*NET的TF-IDF矩阵
result_tfidf = tfidf_matrix[:len(result_df)]
onet_tfidf = tfidf_matrix[len(result_df):]

# 计算余弦相似度
cosine_similarities = cosine_similarity(result_tfidf, onet_tfidf)

# 提取最相似的O*NET职位
matches = []
for idx, row in enumerate(cosine_similarities):
    best_match_idx = row.argmax()  # 获取最大相似度的索引
    best_match_score = row[best_match_idx]  # 获取相似度分数
    best_match_title = onet_jobs_df.iloc[best_match_idx]['Title']  # 获取O*NET职位名称
    best_match_code = onet_jobs_df.iloc[best_match_idx]['Code']  # 获取O*NET职位的代码
    result_code = result_df.iloc[idx]['Code']  # 获取result.csv中的编码
    result_job_name = result_df.iloc[idx]['Title']  # 获取result.csv中的职位名称

    # 保存匹配结果
    matches.append((result_code, result_job_name, best_match_title, best_match_code, best_match_score))

# 将结果保存为DataFrame
matches_df = pd.DataFrame(matches, columns=['Result_Code', 'Result_Job_Name', 'ONet_Job_Title', 'ONet_Job_Code', 'Similarity_Score'])

# 保存结果到CSV文件
matches_df.to_csv("matched_jobs_descriptions_TFIDF_filtered.csv", index=False)

# 打印前几行结果
print(matches_df.head())

  Result_Code                                    Result_Job_Name  \
0  1-01-00-00               Head of the Communist Party of China   
1  1-02-01-00                         Head of state power agency   
2  1-02-02-00                Head of state administrative agency   
3  1-02-03-00  Head of the People's Political Consultative Co...   
4  1-02-04-01                 person in charge of people's court   

                         ONet_Job_Title ONet_Job_Code  Similarity_Score  
0                           Legislators    11-1031.00          0.122229  
1                             Embalmers    39-4011.00          0.094999  
2  Medical and Health Services Managers    11-9111.00          0.127035  
3                             Embalmers    39-4011.00          0.109648  
4                              Bailiffs    33-3011.00          0.288978  


In [5]:
#方法2.1：用SentenceBERT直接对中国职业名的英文翻译和美国O*NET职业名一一匹配
from sentence_transformers import SentenceTransformer
from sklearn.metrics.pairwise import cosine_similarity

# Preview the result file and O*NET job data to check column names
print(result_df.head())
print(onet_jobs_df.head())

# Assuming the 'Title' column in the result file contains the job titles (adjust if necessary)
result_jobs = result_df['Title'].dropna()  # Remove any NaN values from the 'Title' column

# Assuming O*NET job titles are in 'Title' and O*NET job codes are in 'O*NET-SOC Code'
onet_jobs = onet_jobs_df['Title'].dropna()  # Remove any NaN values from O*NET job titles
onet_codes = onet_jobs_df['Code'].dropna()  # The O*NET job codes

# Step 1: Initialize the Sentence-BERT model
model = SentenceTransformer('paraphrase-MiniLM-L6-v2')  # You can also use other models

# Step 2: Encode the job titles into sentence embeddings
result_job_embeddings = model.encode(result_jobs.tolist())  # Encode result jobs into embeddings
onet_job_embeddings = model.encode(onet_jobs.tolist())  # Encode O*NET jobs into embeddings

# Step 3: Compute cosine similarity between result jobs and O*NET jobs
cosine_similarities = cosine_similarity(result_job_embeddings, onet_job_embeddings)

# Step 4: Find the most similar O*NET job for each result job
best_matches = []
for idx, row in enumerate(cosine_similarities):
    best_match_idx = row.argmax()  # Index of the highest cosine similarity
    best_match_score = row[best_match_idx]  # The similarity score
    best_match_onet_job = onet_jobs.iloc[best_match_idx]  # The most similar O*NET job name
    best_match_onet_code = onet_codes.iloc[best_match_idx]  # The O*NET job code
    best_matches.append((result_df.iloc[idx, 0], result_jobs.iloc[idx], best_match_onet_job, best_match_onet_code, best_match_score))

# Step 5: Create a DataFrame for the results
matches_df = pd.DataFrame(best_matches, columns=['Result_Job_Code', 'Result_Job', 'ONet_Job', 'ONet_Code', 'Similarity_Score'])

# 保存结果到CSV文件
matches_df.to_csv("matched_jobs_name_SentenceBERT_filtered.csv", index=False)


         Code ChineseTitle                                     Unnamed: 2  \
0  1-01-00-00   中国共产党机关负责人              在中国共产党中央和地方各级机关及其工作机构中，担任领导职务的人员。   
1  1-02-01-00    国家权力机关负责人      在各级人民代表大会常务委员会及其工作机构中，担任领导职务并具有决策、管理权的人员。   
2  1-02-02-00    国家行政机关负责人           在各级国家行政机关及其工作机构中，担任领导职务并具有决策、管理权的人员。   
3  1-02-03-00    人民政协机关负责人             在各级人民政协及其工作机构中，担任领导职务并具有决策、管理权的人员。   
4  1-02-04-01      人民法院负责人  在最高人民法院、地方各级人民法院以及专门人民法院中，担任领导职务并具有决策、管理权的人员。   

                                               Title  \
0               Head of the Communist Party of China   
1                         Head of state power agency   
2                Head of state administrative agency   
3  Head of the People's Political Consultative Co...   
4                 person in charge of people's court   

                                         Description  \
0  Personnel holding leadership positions in the ...   
1  Personnel who hold leadership positions and ha...   
2  Personnel who hold leadership

In [6]:
#方法2.2：用SentenceBERT根据中美职业定义对中国职业名和美国O*NET职业名一一匹配
from sentence_transformers import SentenceTransformer
from sklearn.metrics.pairwise import cosine_similarity
import pandas as pd
import nltk
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
import string

# 设置 NLTK 数据路径，确保正确加载已下载的资源
nltk.data.path.append(r"C:\Users\for30\AppData\Roaming\nltk_data")  

# 停用词
stop_words = set(stopwords.words('english'))

# 文本预处理函数
def preprocess_text(text):
    if isinstance(text, str):  # 确保是字符串
        # 转小写，去除标点，分词
        text = text.lower()
        tokens = word_tokenize(text)
        # 去除停用词和标点符号
        filtered_tokens = [word for word in tokens if word not in stop_words and word not in string.punctuation]
        return ' '.join(filtered_tokens)
    return ""

# 预处理result.csv和O*NET的Description
result_df['Processed_Description'] = result_df['Description'].apply(preprocess_text)
onet_jobs_df['Processed_Description'] = onet_jobs_df['Description'].apply(preprocess_text)

# 使用Sentence-BERT获取句子嵌入
model = SentenceTransformer('all-MiniLM-L6-v2')  # 选择一个适合的预训练模型

# 获取文本嵌入
result_embeddings = model.encode(result_df['Processed_Description'].tolist())
onet_embeddings = model.encode(onet_jobs_df['Processed_Description'].tolist())

# 计算余弦相似度
cosine_similarities = cosine_similarity(result_embeddings, onet_embeddings)

# 提取最相似的O*NET职位
matches = []
for idx, row in enumerate(cosine_similarities):
    best_match_idx = row.argmax()  # 获取最大相似度的索引
    best_match_score = row[best_match_idx]  # 获取相似度分数
    best_match_title = onet_jobs_df.iloc[best_match_idx]['Title']  # 获取O*NET职位名称
    best_match_code = onet_jobs_df.iloc[best_match_idx]['Code']  # 获取O*NET职位的代码
    result_code = result_df.iloc[idx]['Code']  # 获取result.csv中的编码
    result_job_name = result_df.iloc[idx]['Title']  # 获取result.csv中的职位名称

    # 保存匹配结果
    matches.append((result_code, result_job_name, best_match_title, best_match_code, best_match_score))

# 将结果保存为DataFrame
matches_df = pd.DataFrame(matches, columns=['Result_Code', 'Result_Job_Name', 'ONet_Job_Title', 'ONet_Job_Code', 'Similarity_Score'])

# 保存结果到CSV文件
output_file = "matched_jobs_descriptions_SentenceBERT_filtered.csv"
matches_df.to_csv(output_file, index=False)

# 打印前几行结果
print(matches_df.head())

  Result_Code                                    Result_Job_Name  \
0  1-01-00-00               Head of the Communist Party of China   
1  1-02-01-00                         Head of state power agency   
2  1-02-02-00                Head of state administrative agency   
3  1-02-03-00  Head of the People's Political Consultative Co...   
4  1-02-04-01                 person in charge of people's court   

                                      ONet_Job_Title ONet_Job_Code  \
0  First-Line Supervisors of Office and Administr...    43-1011.00   
1                    General and Operations Managers    11-1021.00   
2                    General and Operations Managers    11-1021.00   
3                    General and Operations Managers    11-1021.00   
4                    General and Operations Managers    11-1021.00   

   Similarity_Score  
0          0.454613  
1          0.552385  
2          0.608470  
3          0.569536  
4          0.471641  


In [7]:
import pandas as pd

# 读取原始 Excel 文件
file_path = "职业库匹配汇总_过滤.xlsx"
df = pd.read_excel(file_path)

# 查看数据的前几行，以确保列名和数据正确
print(df.head())

# Step 1: 删除以 '7-' 或 '8-' 开头的职业代码行
# 确保 '职业代码' 列是字符串类型，避免匹配错误
df['职业代码'] = df['职业代码'].astype(str)
filtered_df = df[~df['职业代码'].str.startswith(('7-', '8-'))]

# Step 2: 将删除后的数据另存为一个新的文件
output_path = "职业库匹配汇总_过滤_去除7和8开头职业代码.xlsx"
filtered_df.to_excel(output_path, index=False)
print(f"Filtered data saved to: {output_path}")

# Step 3: 继续后续处理
# 假设我们继续进行Similarity_Score的处理
score_columns = ['Similarity_Score1', 'Similarity_Score2', 'Similarity_Score3', 'Similarity_Score4']  

# 比较不同的Similarity_Score列，选择最大的值
filtered_df['Similarity_Score_max'] = filtered_df[score_columns].max(axis=1)

# 获取最大Similarity_Score对应的列名
filtered_df['Similarity_Score_method'] = filtered_df[score_columns].idxmax(axis=1).apply(lambda x: x.split('(')[-1].strip(')'))

# 获取最大Similarity_Score对应的 O*Net 职业名称和代码
method_column_map = {
    'Similarity_Score1': ['Job_Title1', 'ONET_code1'],
    'Similarity_Score2': ['Job_Title2', 'ONET_code2'],
    'Similarity_Score3': ['Job_Title3', 'ONET_code3'],
    'Similarity_Score4': ['Job_Title4', 'ONET_code4']
}

# 获取对应列的数据
def get_onet_data(row):
    method = row['Similarity_Score_method']
    job_title_col, onet_code_col = method_column_map.get(method, (None, None))
    if job_title_col and onet_code_col:
        return row[job_title_col], row[onet_code_col]
    return None, None  # 如果没有找到对应的列，返回 None

# 创建新列 'ONET_job' 和 'ONET_code'
filtered_df[['ONET_job', 'ONET_code']] = filtered_df.apply(get_onet_data, axis=1, result_type="expand")

# Step 4: 选择需要的列，并调整顺序
final_df = filtered_df[['职业代码', 'OSTA职业名称', 'OSTA职业名称翻译', 'Similarity_Score_max', 'ONET_job', 'ONET_code', 'Similarity_Score_method']]

# Step 5: 保存为新的 Excel 文件
final_output_path = "matchedjobs最终处理结果_过滤.xlsx"
final_df.to_excel(final_output_path, index=False)

# 打印前几行结果
print(final_df.head())

         职业代码    OSTA职业名称                                         OSTA职业名称翻译  \
0  1-01-00-00  中国共产党机关负责人               Head of the Communist Party of China   
1  1-02-01-00   国家权力机关负责人                         Head of state power agency   
2  1-02-02-00   国家行政机关负责人                Head of state administrative agency   
3  1-02-03-00   人民政协机关负责人  Head of the People's Political Consultative Co...   
4  1-02-04-01     人民法院负责人                 person in charge of people's court   

                             Job_Title1  ONET_code1  Similarity_Score1  \
0                  Chefs and Head Cooks  35-1011.00           0.202686   
1                 Power Plant Operators  51-8013.00           0.252547   
2      Administrative Services Managers  11-3012.00           0.285763   
3                  Political Scientists  19-3094.00           0.350202   
4  Court, Municipal, and License Clerks  43-4031.00           0.259805   

                             Job_Title2  ONET_code2  Similarity_Score2  \


C:\Users\for30\AppData\Local\Temp\ipykernel_4400\2721685683.py:25: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  filtered_df['Similarity_Score_max'] = filtered_df[score_columns].max(axis=1)
C:\Users\for30\AppData\Local\Temp\ipykernel_4400\2721685683.py:28: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  filtered_df['Similarity_Score_method'] = filtered_df[score_columns].idxmax(axis=1).apply(lambda x: x.split('(')[-1].strip(')'))
C:\Users\for30\AppData\Local\Temp\ipykernel_4400\2721685683.py:47: SettingWithCop

         职业代码    OSTA职业名称                                         OSTA职业名称翻译  \
0  1-01-00-00  中国共产党机关负责人               Head of the Communist Party of China   
1  1-02-01-00   国家权力机关负责人                         Head of state power agency   
2  1-02-02-00   国家行政机关负责人                Head of state administrative agency   
3  1-02-03-00   人民政协机关负责人  Head of the People's Political Consultative Co...   
4  1-02-04-01     人民法院负责人                 person in charge of people's court   

   Similarity_Score_max                                           ONET_job  \
0              0.454613  First-Line Supervisors of Office and Administr...   
1              0.552385                    General and Operations Managers   
2              0.697787                   Administrative Services Managers   
3              0.569536                    General and Operations Managers   
4              0.539457  Administrative Law Judges, Adjudicators, and H...   

    ONET_code Similarity_Score_method  
0  43-1011

In [8]:
# 计算每个方法出现的频率
method_frequency = filtered_df['Similarity_Score_method'].value_counts()

# 计算每种相似度方法Similarity_Score的均值（平均值）、中位数、标准差（样本标准差）
mean_values = df[score_columns].mean()
median_values = df[score_columns].median()
std_values = df[score_columns].std()

# 创建一个包含方法频率的 DataFrame
method_frequency_df = method_frequency.reset_index()
method_frequency_df.columns = ['Similarity_Score_method', 'Frequency']

# 将方法频率统计结果与相似度分数统计结果合并
# 创建一个新的 DataFrame 存储统计结果
summary_df = pd.DataFrame({
    'Mean': mean_values,
    'Median': median_values,
    'Standard Deviation': std_values
})

# 使用 pd.concat 将两个 DataFrame 连接起来，注意添加适当的列名
final_df = pd.concat([method_frequency_df, summary_df], axis=1)

# Step 6: 输出到 Python 控制台
print("\n--- Similarity_Score_method出现的频率 ---")
print(method_frequency)
print("\n--- Similarity_Score的统计 ---")
print("均值 (Mean):")
print(mean_values)
print("\n中位数 (Median):")
print(median_values)
print("\n标准差 (Standard Deviation):")
print(std_values)

# 保存为一个新的 Excel 文件
final_df.to_excel("statistics_summary_filtered.xlsx", index=False)


--- Similarity_Score_method出现的频率 ---
Similarity_Score3    2779
Similarity_Score4    1460
Similarity_Score1      10
Name: Similarity_Score_method, dtype: int64

--- Similarity_Score的统计 ---
均值 (Mean):
Similarity_Score1    0.230506
Similarity_Score2    0.185021
Similarity_Score3    0.591857
Similarity_Score4    0.542003
dtype: float64

中位数 (Median):
Similarity_Score1    0.251678
Similarity_Score2    0.172285
Similarity_Score3    0.588409
Similarity_Score4    0.547544
dtype: float64

标准差 (Standard Deviation):
Similarity_Score1    0.179258
Similarity_Score2    0.082941
Similarity_Score3    0.117995
Similarity_Score4    0.085538
dtype: float64


In [12]:
# 读取文件
matched_result_jobs = pd.read_excel(os.path.join(current_dir, "matchedjobs最终处理结果_过滤.xlsx"))
allscores_SML = pd.read_csv(os.path.join(current_dir, "allscores_SML.csv"))

# 去除空格字符（确保合并时不出错）
matched_result_jobs['ONET_code'] = matched_result_jobs['ONET_code'].str.strip()
allscores_SML['O*NET-SOC Code'] = allscores_SML['O*NET-SOC Code'].str.strip()

# 确保 ONet_code 和 O*NET-SOC Code 都是字符串类型
matched_result_jobs['ONET_code'] = matched_result_jobs['ONET_code'].astype(str)
allscores_SML['O*NET-SOC Code'] = allscores_SML['O*NET-SOC Code'].astype(str)

# 第一步：将 matched_result_jobs 与 allscores_SML 合并，获取 mSML、vmSML 和 sdmSML
merged = pd.merge(matched_result_jobs, allscores_SML[['O*NET-SOC Code', 'mSML', 'vmSML', 'sdmSML']], 
                  left_on='ONET_code', right_on='O*NET-SOC Code', how='left')

# 第二步：查看哪些行没有找到匹配项
unmatched = merged[merged['mSML'].isna()]  # 找到没有匹配的行
print("未匹配的数据行：")
print(unmatched[['ONET_code', 'O*NET-SOC Code']])  # 打印未匹配的 ONET_code 和 O*NET-SOC Code

# 将未匹配的结果保存为文件，以供进一步分析
unmatched.to_csv("unmatched_jobs_filtered.csv", index=False, encoding='utf-8-sig')

# 第三步：将结果与 result_df 合并，获取中文职业名称
result_file = os.path.join(current_dir, "result中英版 - 工作表1.csv")
result_df = pd.read_csv(result_file, encoding='gbk')

merged = pd.merge(merged, result_df[['Code', 'ChineseTitle']], 
                  left_on='职业代码', right_on='Code', how='left')

# 第四步：创建最终的 DataFrame，选取所需的列
final_result = merged[['职业代码', 'ChineseTitle','O*NET-SOC Code', 'mSML']]

# 重命名列名
final_result.columns = ['职业代码', '职业名称', 'O*NET-SOC Code', 'SML']

# 保存结果
final_result.to_csv("final_result_filtered.csv", index=False, encoding='utf-8-sig')

# 打印最终结果的前几行
print(final_result.head())

# 读取Occupational Data 
occupation_data_file = os.path.join(current_dir, "filtered_occupation_data.csv")
occupation_data = pd.read_csv(occupation_data_file)
# 计算 unmatched_jobs 中不相同的 ONET_code 数量
unique_unmatched_onet_codes = unmatched['ONET_code'].nunique()
print(f"未匹配的不同的 ONET_code 数量: {unique_unmatched_onet_codes}")

# 获取未匹配的 ONET_code 列
unmatched_onet_codes = unmatched['ONET_code'].unique()

# 从 Occupation Data 中删除这些 ONET_code 对应的行
occupation_data_filtered = occupation_data[~occupation_data['Code'].isin(unmatched_onet_codes)]

# 计算删除的行数
deleted_rows_count = len(occupation_data) - len(occupation_data_filtered)
print(f"删除的行数: {deleted_rows_count}")

# 保存过滤后的 Occupation Data
occupation_data_filtered.to_csv("filtered2_occupation_data.csv", index=False, encoding='utf-8-sig')

未匹配的数据行：
       ONET_code O*NET-SOC Code
265   29-1243.00            NaN
333   29-9021.00            NaN
611   29-9021.00            NaN
1618  53-3053.00            NaN
3159  53-1049.00            NaN
         职业代码        职业名称 O*NET-SOC Code       SML
0  1-01-00-00  中国共产党机关负责人     43-1011.00  3.560483
1  1-02-01-00   国家权力机关负责人     11-1021.00  3.468009
2  1-02-02-00   国家行政机关负责人     11-3012.00  3.363453
3  1-02-03-00   人民政协机关负责人     11-1021.00  3.468009
4  1-02-04-01     人民法院负责人     23-1021.00  3.306054
未匹配的不同的 ONET_code 数量: 4
删除的行数: 4


In [13]:
#方法1.1：用TF-IDF直接对中国职业名的英文翻译和美国O*NET职业名一一匹配
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity

# Load the result.csv file
result_file = os.path.join(current_dir, 'result中英版 - 工作表1.csv')
result_df = pd.read_csv(result_file,encoding='gbk')

# Load the O*NET job data (from CSV file)
onet_file = os.path.join(current_dir, 'filtered2_occupation_data.csv')
onet_jobs_df = pd.read_csv(onet_file)

# Preview the result file and O*NET job data to check column names
print(result_df.head())
print(onet_jobs_df.head())

# Assuming the 'Title' column in the result file contains the job titles (adjust if necessary)
result_jobs = result_df['Title'].dropna()  # Remove any NaN values from the D column

# Assuming O*NET job titles are in 'Job_Title' and O*NET job codes are in 'SOC_code'
onet_jobs = onet_jobs_df['Title'].dropna()  # Remove any NaN values from O*NET job titles
onet_codes = onet_jobs_df['Code'].dropna()  # The O*NET job codes

# Step 1: Create a TF-IDF vectorizer to transform the job titles into vector form
vectorizer = TfidfVectorizer(stop_words='english')

# Step 2: Combine the result job titles and O*NET job titles into one list
all_jobs = list(result_jobs) + list(onet_jobs)

# Step 3: Fit and transform the combined job titles
tfidf_matrix = vectorizer.fit_transform(all_jobs)

# Step 4: Split the TF-IDF matrix into two parts: one for result jobs and one for O*NET jobs
result_tfidf = tfidf_matrix[:len(result_jobs)]
onet_tfidf = tfidf_matrix[len(result_jobs):]

# Step 5: Compute cosine similarity between result jobs and O*NET jobs
cosine_similarities = cosine_similarity(result_tfidf, onet_tfidf)

# Step 6: Find the most similar O*NET job for each result job
best_matches = []
for idx, row in enumerate(cosine_similarities):
    best_match_idx = row.argmax()  # Index of the highest cosine similarity
    best_match_score = row[best_match_idx]  # The similarity score
    best_match_onet_job = onet_jobs.iloc[best_match_idx]  # The most similar O*NET job name
    best_match_onet_code = onet_codes.iloc[best_match_idx]  # The O*NET job code
    best_matches.append((result_df.iloc[idx, 0], result_jobs.iloc[idx], best_match_onet_job, best_match_onet_code, best_match_score))

# Step 7: Create a DataFrame for the results
matches_df = pd.DataFrame(best_matches, columns=['Result_Job_Code', 'Result_Job', 'ONet_Job', 'ONet_Code', 'Similarity_Score'])

# 保存结果到CSV文件
matches_df.to_csv("matched_jobs_name_TFIDF_filtered2.csv", index=False)

# 打印前几行结果
print(matches_df.head())
#方法1.2：用TF-IDF根据中美职业定义对中国职业名和美国O*NET职业名一一匹配
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity
import nltk
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
import string


# 设置 NLTK 数据路径，确保正确加载已下载的资源
nltk.data.path.append(r"C:\Users\for30\AppData\Roaming\nltk_data")  


# 停用词
stop_words = set(stopwords.words('english'))

# 文本预处理函数
def preprocess_text(text):
    if isinstance(text, str):  # 确保是字符串
        # 转小写，去除标点，分词
        text = text.lower()
        tokens = word_tokenize(text)
        # 去除停用词和标点符号
        filtered_tokens = [word for word in tokens if word not in stop_words and word not in string.punctuation]
        return ' '.join(filtered_tokens)
    return ""

# 预处理result.csv和O*NET的Description
result_df['Processed_Description'] = result_df['Description'].apply(preprocess_text)
onet_jobs_df['Processed_Description'] = onet_jobs_df['Description'].apply(preprocess_text)

# 使用TfidfVectorizer转换文本
vectorizer = TfidfVectorizer()
# 合并所有的Description进行向量化
all_descriptions = result_df['Processed_Description'].tolist() + onet_jobs_df['Processed_Description'].tolist()

# 计算TF-IDF矩阵
tfidf_matrix = vectorizer.fit_transform(all_descriptions)

# 分割result和O*NET的TF-IDF矩阵
result_tfidf = tfidf_matrix[:len(result_df)]
onet_tfidf = tfidf_matrix[len(result_df):]

# 计算余弦相似度
cosine_similarities = cosine_similarity(result_tfidf, onet_tfidf)

# 提取最相似的O*NET职位
matches = []
for idx, row in enumerate(cosine_similarities):
    best_match_idx = row.argmax()  # 获取最大相似度的索引
    best_match_score = row[best_match_idx]  # 获取相似度分数
    best_match_title = onet_jobs_df.iloc[best_match_idx]['Title']  # 获取O*NET职位名称
    best_match_code = onet_jobs_df.iloc[best_match_idx]['Code']  # 获取O*NET职位的代码
    result_code = result_df.iloc[idx]['Code']  # 获取result.csv中的编码
    result_job_name = result_df.iloc[idx]['Title']  # 获取result.csv中的职位名称

    # 保存匹配结果
    matches.append((result_code, result_job_name, best_match_title, best_match_code, best_match_score))

# 将结果保存为DataFrame
matches_df = pd.DataFrame(matches, columns=['Result_Code', 'Result_Job_Name', 'ONet_Job_Title', 'ONet_Job_Code', 'Similarity_Score'])

# 保存结果到CSV文件
matches_df.to_csv("matched_jobs_descriptions_TFIDF_filtered2.csv", index=False)

# 打印前几行结果
print(matches_df.head())
#方法2.1：用SentenceBERT直接对中国职业名的英文翻译和美国O*NET职业名一一匹配
from sentence_transformers import SentenceTransformer
from sklearn.metrics.pairwise import cosine_similarity

# Preview the result file and O*NET job data to check column names
print(result_df.head())
print(onet_jobs_df.head())

# Assuming the 'Title' column in the result file contains the job titles (adjust if necessary)
result_jobs = result_df['Title'].dropna()  # Remove any NaN values from the 'Title' column

# Assuming O*NET job titles are in 'Title' and O*NET job codes are in 'O*NET-SOC Code'
onet_jobs = onet_jobs_df['Title'].dropna()  # Remove any NaN values from O*NET job titles
onet_codes = onet_jobs_df['Code'].dropna()  # The O*NET job codes

# Step 1: Initialize the Sentence-BERT model
model = SentenceTransformer('paraphrase-MiniLM-L6-v2')  # You can also use other models

# Step 2: Encode the job titles into sentence embeddings
result_job_embeddings = model.encode(result_jobs.tolist())  # Encode result jobs into embeddings
onet_job_embeddings = model.encode(onet_jobs.tolist())  # Encode O*NET jobs into embeddings

# Step 3: Compute cosine similarity between result jobs and O*NET jobs
cosine_similarities = cosine_similarity(result_job_embeddings, onet_job_embeddings)

# Step 4: Find the most similar O*NET job for each result job
best_matches = []
for idx, row in enumerate(cosine_similarities):
    best_match_idx = row.argmax()  # Index of the highest cosine similarity
    best_match_score = row[best_match_idx]  # The similarity score
    best_match_onet_job = onet_jobs.iloc[best_match_idx]  # The most similar O*NET job name
    best_match_onet_code = onet_codes.iloc[best_match_idx]  # The O*NET job code
    best_matches.append((result_df.iloc[idx, 0], result_jobs.iloc[idx], best_match_onet_job, best_match_onet_code, best_match_score))

# Step 5: Create a DataFrame for the results
matches_df = pd.DataFrame(best_matches, columns=['Result_Job_Code', 'Result_Job', 'ONet_Job', 'ONet_Code', 'Similarity_Score'])

# 保存结果到CSV文件
matches_df.to_csv("matched_jobs_name_SentenceBERT_filtered2.csv", index=False)
#方法2.2：用SentenceBERT根据中美职业定义对中国职业名和美国O*NET职业名一一匹配
from sentence_transformers import SentenceTransformer
from sklearn.metrics.pairwise import cosine_similarity
import pandas as pd
import nltk
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
import string

# 设置 NLTK 数据路径，确保正确加载已下载的资源
nltk.data.path.append(r"C:\Users\for30\AppData\Roaming\nltk_data")  

# 停用词
stop_words = set(stopwords.words('english'))

# 文本预处理函数
def preprocess_text(text):
    if isinstance(text, str):  # 确保是字符串
        # 转小写，去除标点，分词
        text = text.lower()
        tokens = word_tokenize(text)
        # 去除停用词和标点符号
        filtered_tokens = [word for word in tokens if word not in stop_words and word not in string.punctuation]
        return ' '.join(filtered_tokens)
    return ""

# 预处理result.csv和O*NET的Description
result_df['Processed_Description'] = result_df['Description'].apply(preprocess_text)
onet_jobs_df['Processed_Description'] = onet_jobs_df['Description'].apply(preprocess_text)

# 使用Sentence-BERT获取句子嵌入
model = SentenceTransformer('all-MiniLM-L6-v2')  # 选择一个适合的预训练模型

# 获取文本嵌入
result_embeddings = model.encode(result_df['Processed_Description'].tolist())
onet_embeddings = model.encode(onet_jobs_df['Processed_Description'].tolist())

# 计算余弦相似度
cosine_similarities = cosine_similarity(result_embeddings, onet_embeddings)

# 提取最相似的O*NET职位
matches = []
for idx, row in enumerate(cosine_similarities):
    best_match_idx = row.argmax()  # 获取最大相似度的索引
    best_match_score = row[best_match_idx]  # 获取相似度分数
    best_match_title = onet_jobs_df.iloc[best_match_idx]['Title']  # 获取O*NET职位名称
    best_match_code = onet_jobs_df.iloc[best_match_idx]['Code']  # 获取O*NET职位的代码
    result_code = result_df.iloc[idx]['Code']  # 获取result.csv中的编码
    result_job_name = result_df.iloc[idx]['Title']  # 获取result.csv中的职位名称

    # 保存匹配结果
    matches.append((result_code, result_job_name, best_match_title, best_match_code, best_match_score))

# 将结果保存为DataFrame
matches_df = pd.DataFrame(matches, columns=['Result_Code', 'Result_Job_Name', 'ONet_Job_Title', 'ONet_Job_Code', 'Similarity_Score'])

# 保存结果到CSV文件
output_file = "matched_jobs_descriptions_SentenceBERT_filtered2.csv"
matches_df.to_csv(output_file, index=False)

# 打印前几行结果
print(matches_df.head())



         Code ChineseTitle                                     Unnamed: 2  \
0  1-01-00-00   中国共产党机关负责人              在中国共产党中央和地方各级机关及其工作机构中，担任领导职务的人员。   
1  1-02-01-00    国家权力机关负责人      在各级人民代表大会常务委员会及其工作机构中，担任领导职务并具有决策、管理权的人员。   
2  1-02-02-00    国家行政机关负责人           在各级国家行政机关及其工作机构中，担任领导职务并具有决策、管理权的人员。   
3  1-02-03-00    人民政协机关负责人             在各级人民政协及其工作机构中，担任领导职务并具有决策、管理权的人员。   
4  1-02-04-01      人民法院负责人  在最高人民法院、地方各级人民法院以及专门人民法院中，担任领导职务并具有决策、管理权的人员。   

                                               Title  \
0               Head of the Communist Party of China   
1                         Head of state power agency   
2                Head of state administrative agency   
3  Head of the People's Political Consultative Co...   
4                 person in charge of people's court   

                                         Description  
0  Personnel holding leadership positions in the ...  
1  Personnel who hold leadership positions and ha...  
2  Personnel who hold leadership po

In [14]:
# 读取原始 Excel 文件
file_path = "职业库匹配汇总_过滤2.xlsx"
df = pd.read_excel(file_path)

# 查看数据的前几行，以确保列名和数据正确
print(df.head())

# Step 1: 删除以 '7-' 或 '8-' 开头的职业代码行
# 确保 '职业代码' 列是字符串类型，避免匹配错误
df['职业代码'] = df['职业代码'].astype(str)
filtered_df = df[~df['职业代码'].str.startswith(('7-', '8-'))]

# Step 2: 将删除后的数据另存为一个新的文件
output_path = "职业库匹配汇总_过滤2_去除7和8开头职业代码.xlsx"
filtered_df.to_excel(output_path, index=False)
print(f"Filtered data saved to: {output_path}")

# Step 3: 继续后续处理
# 假设我们继续进行Similarity_Score的处理
score_columns = ['Similarity_Score1', 'Similarity_Score2', 'Similarity_Score3', 'Similarity_Score4']  

# 比较不同的Similarity_Score列，选择最大的值
filtered_df['Similarity_Score_max'] = filtered_df[score_columns].max(axis=1)

# 获取最大Similarity_Score对应的列名
filtered_df['Similarity_Score_method'] = filtered_df[score_columns].idxmax(axis=1).apply(lambda x: x.split('(')[-1].strip(')'))

# 获取最大Similarity_Score对应的 O*Net 职业名称和代码
method_column_map = {
    'Similarity_Score1': ['Job_Title1', 'ONET_code1'],
    'Similarity_Score2': ['Job_Title2', 'ONET_code2'],
    'Similarity_Score3': ['Job_Title3', 'ONET_code3'],
    'Similarity_Score4': ['Job_Title4', 'ONET_code4']
}

# 获取对应列的数据
def get_onet_data(row):
    method = row['Similarity_Score_method']
    job_title_col, onet_code_col = method_column_map.get(method, (None, None))
    if job_title_col and onet_code_col:
        return row[job_title_col], row[onet_code_col]
    return None, None  # 如果没有找到对应的列，返回 None

# 创建新列 'ONET_job' 和 'ONET_code'
filtered_df[['ONET_job', 'ONET_code']] = filtered_df.apply(get_onet_data, axis=1, result_type="expand")

# Step 4: 选择需要的列，并调整顺序
final_df = filtered_df[['职业代码', 'OSTA职业名称', 'OSTA职业名称翻译', 'Similarity_Score_max', 'ONET_job', 'ONET_code', 'Similarity_Score_method']]

# Step 5: 保存为新的 Excel 文件
final_output_path = "matchedjobs最终处理结果_过滤2.xlsx"
final_df.to_excel(final_output_path, index=False)

# 打印前几行结果
print(final_df.head())
# 计算每个方法出现的频率
method_frequency = filtered_df['Similarity_Score_method'].value_counts()

# 计算每种相似度方法Similarity_Score的均值（平均值）、中位数、标准差（样本标准差）
mean_values = df[score_columns].mean()
median_values = df[score_columns].median()
std_values = df[score_columns].std()

# 创建一个包含方法频率的 DataFrame
method_frequency_df = method_frequency.reset_index()
method_frequency_df.columns = ['Similarity_Score_method', 'Frequency']

# 将方法频率统计结果与相似度分数统计结果合并
# 创建一个新的 DataFrame 存储统计结果
summary_df = pd.DataFrame({
    'Mean': mean_values,
    'Median': median_values,
    'Standard Deviation': std_values
})

# 使用 pd.concat 将两个 DataFrame 连接起来，注意添加适当的列名
final_df = pd.concat([method_frequency_df, summary_df], axis=1)

# Step 6: 输出到 Python 控制台
print("\n--- Similarity_Score_method出现的频率 ---")
print(method_frequency)
print("\n--- Similarity_Score的统计 ---")
print("均值 (Mean):")
print(mean_values)
print("\n中位数 (Median):")
print(median_values)
print("\n标准差 (Standard Deviation):")
print(std_values)

# 保存为一个新的 Excel 文件
final_df.to_excel("statistics_summary_filtered2.xlsx", index=False)
# 读取文件
matched_result_jobs = pd.read_excel(os.path.join(current_dir, "matchedjobs最终处理结果_过滤2.xlsx"))
allscores_SML = pd.read_csv(os.path.join(current_dir, "allscores_SML.csv"))

# 去除空格字符（确保合并时不出错）
matched_result_jobs['ONET_code'] = matched_result_jobs['ONET_code'].str.strip()
allscores_SML['O*NET-SOC Code'] = allscores_SML['O*NET-SOC Code'].str.strip()

# 确保 ONet_code 和 O*NET-SOC Code 都是字符串类型
matched_result_jobs['ONET_code'] = matched_result_jobs['ONET_code'].astype(str)
allscores_SML['O*NET-SOC Code'] = allscores_SML['O*NET-SOC Code'].astype(str)

# 第一步：将 matched_result_jobs 与 allscores_SML 合并，获取 mSML、vmSML 和 sdmSML
merged = pd.merge(matched_result_jobs, allscores_SML[['O*NET-SOC Code', 'mSML', 'vmSML', 'sdmSML']], 
                  left_on='ONET_code', right_on='O*NET-SOC Code', how='left')

# 第二步：查看哪些行没有找到匹配项
unmatched = merged[merged['mSML'].isna()]  # 找到没有匹配的行
print("未匹配的数据行：")
print(unmatched[['ONET_code', 'O*NET-SOC Code']])  # 打印未匹配的 ONET_code 和 O*NET-SOC Code

# 将未匹配的结果保存为文件，以供进一步分析
unmatched.to_csv("unmatched_jobs_filtered2.csv", index=False, encoding='utf-8-sig')

# 第三步：将结果与 result_df 合并，获取中文职业名称
result_file = os.path.join(current_dir, "result中英版 - 工作表1.csv")
result_df = pd.read_csv(result_file, encoding='gbk')

merged = pd.merge(merged, result_df[['Code', 'ChineseTitle']], 
                  left_on='职业代码', right_on='Code', how='left')

# 第四步：创建最终的 DataFrame，选取所需的列
final_result = merged[['职业代码', 'ChineseTitle','O*NET-SOC Code', 'mSML']]

# 重命名列名
final_result.columns = ['职业代码', '职业名称', 'O*NET-SOC Code', 'SML']

# 保存结果
final_result.to_csv("final_result_filtered2.csv", index=False, encoding='utf-8-sig')

# 打印最终结果的前几行
print(final_result.head())



         职业代码    OSTA职业名称                                         OSTA职业名称翻译  \
0  1-01-00-00  中国共产党机关负责人               Head of the Communist Party of China   
1  1-02-01-00   国家权力机关负责人                         Head of state power agency   
2  1-02-02-00   国家行政机关负责人                Head of state administrative agency   
3  1-02-03-00   人民政协机关负责人  Head of the People's Political Consultative Co...   
4  1-02-04-01     人民法院负责人                 person in charge of people's court   

                             Job_Title1  ONET_code1  Similarity_Score1  \
0                  Chefs and Head Cooks  35-1011.00           0.202678   
1                 Power Plant Operators  51-8013.00           0.252543   
2      Administrative Services Managers  11-3012.00           0.285762   
3                  Political Scientists  19-3094.00           0.350205   
4  Court, Municipal, and License Clerks  43-4031.00           0.259806   

                             Job_Title2  ONET_code2  Similarity_Score2  \


C:\Users\for30\AppData\Local\Temp\ipykernel_4400\4216195786.py:23: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  filtered_df['Similarity_Score_max'] = filtered_df[score_columns].max(axis=1)
C:\Users\for30\AppData\Local\Temp\ipykernel_4400\4216195786.py:26: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  filtered_df['Similarity_Score_method'] = filtered_df[score_columns].idxmax(axis=1).apply(lambda x: x.split('(')[-1].strip(')'))
C:\Users\for30\AppData\Local\Temp\ipykernel_4400\4216195786.py:45: SettingWithCop

         职业代码    OSTA职业名称                                         OSTA职业名称翻译  \
0  1-01-00-00  中国共产党机关负责人               Head of the Communist Party of China   
1  1-02-01-00   国家权力机关负责人                         Head of state power agency   
2  1-02-02-00   国家行政机关负责人                Head of state administrative agency   
3  1-02-03-00   人民政协机关负责人  Head of the People's Political Consultative Co...   
4  1-02-04-01     人民法院负责人                 person in charge of people's court   

   Similarity_Score_max                                           ONET_job  \
0              0.454613  First-Line Supervisors of Office and Administr...   
1              0.552385                    General and Operations Managers   
2              0.697787                   Administrative Services Managers   
3              0.569536                    General and Operations Managers   
4              0.539457  Administrative Law Judges, Adjudicators, and H...   

    ONET_code Similarity_Score_method  
0  43-1011